In [ ]:
! pip install langchain-anthropic langgraph

In [1]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("ANTHROPIC_API_KEY")

## Instructions

You can think of this as "procedural" memory. 

These are the instructions that the LLM will follow to extract memories from the journal entry.

First, we define a general schema for any memory.

The schema has a `memory_type` and a `memory_content`.

![](./img/memory_course_procedural.png)


In [3]:
from typing import List
from pydantic import BaseModel, Field
from langchain_anthropic import ChatAnthropic

# Schema for a memory
class Memory(BaseModel):
    memory_type: str = Field(None, description="Type of memory to extract.")
    memory_content: str = Field(None, description="Specific content of the memory.")

# List of memories 
class Memories(BaseModel):
    memories: List[Memory] = Field(None, description="List of memories to extract.")

# Define and augment LLM with structured output
llm = ChatAnthropic(model="claude-3-5-sonnet-latest")
structured_llm = llm.with_structured_output(Memories)

Now let's load our instructions for memory extraction.

In [14]:
import sys
sys.path.append('..')
import src.memory_course.prompts

# Reload the module in case any changes were made
import importlib
importlib.reload(src.memory_course.prompts)

# Then import the instructions 
from src.memory_course.prompts import memory_collection_extraction_instructions, update_memory_collection_extraction_instructions

Now, we want to store these in a way that our application can easily access to modify them. 
 
The [LangGraph store](https://docs.langchain.com/docs/components/store/memory) is a simple key-value store that we can use for long-term storage. 

We get it for "free" with any LangGraph deployment (local or hosted).

In [15]:
# Save instructions to the store
from langgraph.store.memory import InMemoryStore
in_memory_store = InMemoryStore()
namespace = ("journal","instructions")
key = "instructions_extraction"
in_memory_store.put(namespace, key, {"instructions": memory_collection_extraction_instructions})

Now, we can create a function that will extract memories from a journal entry.

In [16]:
from langchain_core.messages import HumanMessage, SystemMessage

def extract_memories(journal_entry: str) -> Memories:
    """Extract memories from a journal entry.
    
    This function analyzes a journal entry and extracts memories
    
    Args:
        journal_entry (str): The text of the journal entry to analyze
        
    Returns:
        Memories (list): A structured list of Memory objects"""

    # Get the instructions from the store
    namespace = ("journal","instructions")
    key = "instructions_extraction"
    procedural_memory = in_memory_store.get(namespace, key)

    # Extract memories
    memories = structured_llm.invoke([SystemMessage(content=procedural_memory.value['instructions']),
                                      HumanMessage(content=f"Extract memory from <context> {journal_entry} </context>")])
    
    return memories

Now, we can test extraction from a fake journal entry.


In [7]:
journal_entry = """
Today was a mix of ups and downs. Started the morning feeling energized after my new morning routine, but got a bit overwhelmed with the project deadlines in the afternoon. 

Need to remember to:
- Email Sarah about the client presentation by Thursday
- Schedule dentist appointment
- Pick up groceries for weekend dinner party

Had an interesting thought during my walk - what if we incorporated AI into our customer feedback system? Could potentially automate the initial response and categorization. Would need to research existing solutions first.

Despite the stress, I'm optimistic about where things are heading. The team's really coming together on the new initiative."""

# Run extraction
memories = extract_memories(journal_entry)

# Review
for memory in memories.memories:
    print("=== Extracted Memory ===")
    print(f"\n{memory.memory_type.upper()}:")
    print(f"- {memory.memory_content}\n")


=== Extracted Memory ===

SENTIMENT:
- Mixed emotions with overall optimistic tone. Started energized, experienced afternoon overwhelm with deadlines, but ended positively about team progress. Key phrases: 'energized', 'overwhelmed', 'optimistic', 'team's coming together'.

=== Extracted Memory ===

TODO:
- Email Sarah about the client presentation (Deadline: Thursday)

=== Extracted Memory ===

TODO:
- Schedule dentist appointment

=== Extracted Memory ===

TODO:
- Pick up groceries for weekend dinner party

=== Extracted Memory ===

IDEA:
- Implement AI in customer feedback system for automated initial response and categorization. Required: Research existing solutions first.



In a journal, we want the ability to update instructions directly based upon user feedback. 

For example, above notice that we create `IDEA:` memories with a `Required` prerequisite:

```
IDEA:
- Implement AI in customer feedback system for automated initial response and categorization. Required: Research existing solutions first.
```

Let's assume that we don't want this.

We can easily udpate our instructions, simply by fetching the current instructions from the store and updating them.

In [10]:
# Function for updating the instructions
def update_extract_memories_instructions(user_feedback: str) -> str:
    """Update instructions for memory extraction based on user feedback.
        
    Args:
        user_feedback (str): User feedback about the memory extraction process
        
    Returns:
        None"""

    # Get the current instructions
    namespace = ("journal","instructions")
    key = "instructions_extraction"
    instructions = in_memory_store.get(namespace, key)

    # Extract memories
    updated_instructions = llm.invoke([SystemMessage(content=update_memory_collection_extraction_instructions.format(instructions=instructions.value['instructions'])),
                                            HumanMessage(content=f"<Feedback> {user_feedback} </Feedback>")])
    
    # Add the new instructions to the store
    in_memory_store.put(namespace, key, {"instructions": updated_instructions.content})

update_extract_memories_instructions("""When saving ideas dont include Required or Prerequisites.""")

In [11]:
# Run extraction
memories = extract_memories(journal_entry)

# Review
for memory in memories.memories:
    print("=== Extracted Memory ===")
    print(f"\n{memory.memory_type.upper()}:")
    print(f"- {memory.memory_content}\n")

=== Extracted Memory ===

SENTIMENT:
- Mixed emotions with overall optimistic tone. Started energized, experienced afternoon overwhelm/stress, ended with positive outlook. Key phrases: 'feeling energized', 'got overwhelmed', 'optimistic about where things are heading'. Notable positive mention of team cohesion.

=== Extracted Memory ===

TODO:
- Email Sarah about client presentation - Due Thursday

=== Extracted Memory ===

TODO:
- Schedule dentist appointment

=== Extracted Memory ===

TODO:
- Pick up groceries for weekend dinner party

=== Extracted Memory ===

IDEA:
- Integrate AI into customer feedback system to automate initial response and categorization



We ca see that the `Required` prerequisite is no longer present:

```
IDEA:
- Integrate AI into customer feedback system to automate initial response and categorization
````

We can see that this is because the instructions were updated:

In [13]:
# Get the current instructions
procedural_memory = in_memory_store.get(namespace, key)
procedural_memory.value['instructions']

'<Instructions>\nGiven a journal entry, analyze and extract the following elements:\n\n1. SENTIMENT:\n   - Identify the overall emotional tone (positive, negative, neutral)\n   - Note any significant emotional shifts or patterns\n   - Extract key emotional words or phrases\n   Output as a single memory with memory_type "SENTIMENT"\n\n2. TODOS:\n   - Extract each task, action, or commitment as a SEPARATE memory\n   - Each TODO should be its own Memory object with memory_type "TODO"\n   - Include deadline in the memory_content if mentioned\n   - Format each todo concisely and clearly\n\n3. IDEAS:\n   - Extract each creative thought, insight, or potential project as a SEPARATE memory\n   - Each idea should be its own Memory object with memory_type "IDEA"\n   - Keep each idea focused and specific\n   - Include only the core concept and relevant details, without listing requirements or prerequisites\n\nImportant: Create a new Memory object for EACH individual todo and idea. Do not combine m

So now we can:

1. Extract memories from a journal entry given a set of instructions (system message)
2. Update those instructions based upon user feedback directly 

This is a simple example of using and editing "procedural memory". 

In the next lesson, we will look at how to add in "episodic memory" or few-shot-example to improve the quality of the extraction.